In [ ]:
%matplotlib inline  
import pandas as pd #data manipulation
import numpy as np #numerical calculations
import matplotlib.pyplot as plt #plot
import yfinance as yf  #幫我們抓取分析資料
import datetime #時間套件
from ipywidgets import interact
from ipywidgets import interact_manual
pd.options.mode.chained_assignment = None  # default='warn' 
#安裝套件指令
#!pip install yfinance  
#!pip install datetime  

##自由發揮題：互動式模擬交易機器人

* 目的：設計一個客製化的"互動式模擬交易機器人"，讓沒有程式基礎的使用者能夠透過簡單的界面輕鬆的選擇想進行交易的標的，自由的更改交易策略參數。期望能夠幫助使用者建立一個能夠跑贏'買進持有'(Buy and Hold)策略的交易策略。

* 交易策略類型：順勢交易-均線交叉策略
* 策略說明：由於移動平均線上升時，價格通常會上升趨勢；移動平均線下降時，價格通常為下跌趨勢。  
故順勢交易策略設計如下：  
買入時機：價格由移動平均線下方突破移動平均線時；當短均線向上交叉長均線。  
賣出時機：價格由移動平均線上方跌破移動平均線時；當短均線向下交叉長均線。  
* 標的：使用者可以自由選定！只需輸入股票代號即可。  
* 交易參數：短線以及長線參數均由使用者自訂。
手續費預設為交易金額的0.01%。
* 資料區間：2020/01/01-2022/01/01，我們先拿過去兩年資料做測試吧。   
* 資料來源：Yahoo Finance



 


### 來試看看是否能建立出比單純Buy& Hold更好的交易策略吧！




In [ ]:
def trade(股票代號, 短均線, 長均線): 

    short_window = int(短均線)
    long_window = int(長均線)
    if (short_window >= long_window):
        return '長均線參數必須比短均線大喔！'
    fee=0.0001
    stock = yf.download(str(股票代號),
                  start=datetime.datetime(2020, 1, 1),
                  end=datetime.datetime(2022, 1, 1))
    
    stock = stock.dropna()
    #初始化交易訊號
    signals = pd.DataFrame(index=stock.index) 
    signals['signal'] = 0
    
    # If window length is 0, algorithm doesn't make sense, so exit
    if (long_window == 0) or (short_window == 0):
      return 0

    # Compute rolling mean and rolling standard deviation
    #短期均線
    signals['short_mavg'] = np.round(stock['Adj Close'].rolling(window=short_window).mean(), 2)  # np.round 四捨五入到小數後2位
    #長期均線
    signals['long_mavg'] = np.round(stock['Adj Close'].rolling(window=long_window).mean(), 2)

    # 交易策略制定
    # Prerequisite:Start with no money and no positions
    
    #產生交易訊號，訊號產生方式 短均線突破長均線=>Buy=>signal = 1
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1, 0)
    #加上 [short_window:] 是為了遵守條件“僅針對大於最短移動均值視窗的時期”
    
    # Generate trading orders
    signals['positions'] = signals['signal'].diff() #positions = 1 代表現在買入動作

    #每天日報酬率
    signals['D_Ret'] =stock['Adj Close'] / stock['Adj Close'].shift(1)-1   
    #實際持有部位時才計算股票日報酬率
    signals['D_RetMod'] = np.where(signals['signal']==1  , signals['D_Ret'], 0) #持有部位時(signal = 1)的當天日報酬率
    #實際持有股票時+採用策略後的累積報酬率
    signals['D_RetMod-1']= np.where(signals['positions']==1  , signals['D_Ret'], 0) #買入動作(positions = 1)當日報酬率
    #持有部位時(signal = 1)的當天日報酬率，買進日跟賣出日當天positions = 1 OR -1)的報酬率不計算
    signals['D_RetMod-2']= signals['D_RetMod']-signals['D_RetMod-1']

    #考慮手續費，只有買入算一次
    signals['D_RetMod2'] = np.where(signals['positions']==1 ,  signals['D_RetMod-2']-fee, signals['D_RetMod-2'])
    
    #做策略修正，因為交易訊號透過收盤價判斷，賣出部位訊號要修改至隔天，隔天才能將部位賣出
    signals['D_RetMod3'] = np.where(signals['positions']== -1 ,signals['D_Ret'], signals['D_RetMod2'])  
 
    # 計算累積報酬率(績效) # .cumprod()累積乘積 
    signals['CumRetM'] = (1 + signals['D_RetMod3']).cumprod()  #此交易策略績效
    signals['Buy&Hold'] = (1 + signals['D_Ret']).cumprod() #一開始Buy and Hold 績效

 
    #畫圖比較績效

    #exclude a few columns from a DataFrame plot
    df_compare = signals.loc[:,['CumRetM','Buy&Hold']]
    (df_compare-1).plot(figsize=(15,7), grid=True,ylabel='Return')
    plt.title(str(股票代號))
    plt.show()
    #最佳化後,短均線參數 = 36日,長均線參數 = 76日
        

In [ ]:
interact_manual(trade,股票代號 = '請輸入股票代號' ,短均線 = range(1,200), 長均線 = range(2,200))

interactive(children=(Text(value='請輸入股票代號', description='股票代號'), Dropdown(description='短均線', options=(1, 2, 3,…

<function __main__.trade>

### 最後，股票交易和投資具有一定風險，交易人應先評估本身資金及所能擔負之風險，過去績效或未來預期的表現不可作為日後績效之保證。